# Stellar Blends Classification

### In this notebook we run the un-normalized and normalized datasets through the MuyGPyS classifier (a python classifying function that uses the MuyGPS  Gaussian process hyperparameter estimation method), and compare the resulting accuracies.

**Note:** Must have run `data_normalization.ipynb` to continue.

In [15]:
# from MuyGPyS import config
# config.update("muygpys_jax_enabled", False)

import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from MuyGPyS.examples.classify import do_classify
from MuyGPyS.gp.deformation import F2, Isotropy
from MuyGPyS.gp.hyperparameter import Parameter, Parameter as ScalarParam
from MuyGPyS.gp.kernels import RBF, Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.loss import LossFn, cross_entropy_fn



### Read in all flattened data (normalized and un-normalized):

In [16]:
from glob import glob

# read normalized data csv file names from the data directory
norm_data_names = glob('../data/data-norm/max-pixel-all/*.csv')
# get rid of "../data/data-norm/max-pixel-all/"
norm_data_names = [name.split('/')[-1] for name in norm_data_names]
# norm_data_names[:10]

In [17]:
# sort the names by their numbers
norm_data_names.sort(key=lambda x: x.split('_')[1])
norm_data_names[:10]

['nthroot_0.0_data.csv',
 'nthroot_0.03448_data.csv',
 'nthroot_0.03448.csv',
 'nthroot_0.06897_data.csv',
 'nthroot_0.06897.csv',
 'nthroot_0.1034_data.csv',
 'nthroot_0.1034.csv',
 'nthroot_0.1379_data.csv',
 'nthroot_0.1379.csv',
 'nthroot_0.1724_data.csv']

### Define a function that generates "one-hot" values.

This essentially just takes our truth labels of 0 and 1, and does the following conversions for use in the classifier:
- 0 to [1., -1.]
- 1 to [-1., 1.]

In [18]:
def generate_onehot_value(values):
    onehot = []
    for val in values:
        if val == 0:
            onehot.append([1., -1.])
        elif val == 1:
            onehot.append([-1., 1.])
    return onehot

### Run the classifier on each dataset

For each dataset (un-normalized and normalized) in `data_files`, this for loop does the following:
- Separate labels from data
- Split up data between training and testing
    - `test_size` is the fraction of the data you want to use for testing, where 0.5 means half of the data is used for testing and half for training.
    - `random_state` makes each dataset get trained and tested on the same number of stars and galaxies.
- Gets the one-hot values for the testing and training labels
- Gets `train` and `test` into the proper format for the classifier, a dictionary with the keys: 
    - 'input': 
    - 'output':
    - 'lookup':
- Does the classification (`do_classify`)
- Computes the accuracy of the classifier for the given dataset, by compairing predicted labels to truth labels.

In [19]:
nn_kwargs_exact = {"nn_method": "exact", "algorithm": "ball_tree"}

nn_kwargs_hnsw = {"nn_method": "hnsw"}

k_kwargs_rbf ={
            "kernel": RBF(
                 deformation=Isotropy(
                     metric=F2,
                 length_scale=Parameter(1.0, (1e-2, 1e2)),
                 ),
            ),
            "noise": HomoscedasticNoise(1e-5),
            }
k_kwargs_mattern= { "kernel": Matern(
             smoothness=ScalarParam(0.5),
             deformation=Isotropy(
                 metric=F2,
                 length_scale=Parameter(1.0, (1e-2, 1e2)),
             ),
         ),
         "noise": HomoscedasticNoise(1e-5),
         }

In [20]:
norm_name = []
my_accuracy = []
for path in tqdm(norm_data_names):
    path1 = '../data/data-norm/max-pixel-all/' + path
    data = pd.read_csv(path1,na_values='-')
    data.fillna(0,inplace=True)
    data_label = ''.join(path.split('.')[:2])
    truth_labels = data.iloc[:, 0].values
    image_data = data.iloc[:, 1:].values

    X_train, X_test, y_train, y_test = train_test_split(image_data, truth_labels, test_size=0.2, random_state=42)

    print("=============== ", data_label, " ===============")
    print('Training data:', len(y_train[y_train==0]), 'single stars and', len(y_train[y_train==1]), 'blended stars')
    print('Testing data:', len(y_test[y_test==0]), 'single stars and', len(y_test[y_test==1]), 'blended stars')

    onehot_train, onehot_test = generate_onehot_value(y_train), generate_onehot_value(y_test)

    train = {'input': X_train, 'output': onehot_train, 'lookup': y_train}
    test = {'input': X_test, 'output': onehot_test, 'lookup': y_test}

    print("Running Classifier on", data_label)
    #Switch verbose to True for more output


    muygps, nbrs_lookup, surrogate_predictions = do_classify(
                                test_features=np.array(test['input']), 
                                train_features=np.array(train['input']), 
                                train_labels=np.array(train['output']), 
                                nn_count=15,
                                batch_count=200,
                                loss_fn=cross_entropy_fn,
                                opt_fn=Bayes_optimize,
                                k_kwargs=k_kwargs_mattern,
                                nn_kwargs=nn_kwargs_hnsw,
                                verbose=False)
    predicted_labels = np.argmax(surrogate_predictions, axis=1)
    accur = np.around((np.sum(predicted_labels == np.argmax(test["output"], axis=1))/len(predicted_labels))*100, 3)
    norm_name.append(''.join(data_label.split('_')[-3:]))
    my_accuracy.append(accur)
    print("Total accuracy for", data_label, ":", accur, '%')

  0%|          | 0/79 [00:00<?, ?it/s]

===============  nthroot_00_data  ===============
Training data: 12072 single stars and 9729 blended stars
Testing data: 3037 single stars and 2414 blended stars
Running Classifier on nthroot_00_data


  1%|▏         | 1/79 [00:02<02:49,  2.17s/it]

Total accuracy for nthroot_00_data : 55.715 %
===============  nthroot_003448_data  ===============
Training data: 12072 single stars and 9729 blended stars
Testing data: 3037 single stars and 2414 blended stars
Running Classifier on nthroot_003448_data


  3%|▎         | 2/79 [00:04<02:43,  2.12s/it]

...

Total accuracy for norm_51csv : 68.043 %
===============  nthroot_log_datacsv  ===============
Training data: 12072 single stars and 9729 blended stars
Testing data: 3037 single stars and 2414 blended stars
Running Classifier on nthroot_log_datacsv


 99%|█████████▊| 78/79 [02:34<00:02,  2.05s/it]

Total accuracy for nthroot_log_datacsv : 77.747 %
===============  nthroot_logcsv  ===============
Training data: 12122 single stars and 9679 blended stars
Testing data: 2987 single stars and 2464 blended stars
Running Classifier on nthroot_logcsv


100%|██████████| 79/79 [02:36<00:00,  1.98s/it]

Total accuracy for nthroot_logcsv : 77.27 %


In [21]:
accura = pd.DataFrame({'norm_name': norm_name, 'accuracy': my_accuracy})
accura.to_csv('../data/muygps-max-all-accuracy.csv', index=False)

accura = pd.read_csv('../data/muygps-max-all-accuracy.csv')   
accura.sort_values(by=['accuracy'], inplace=True)
accura.T

,0,76,67,71,72,53,50,54,68,36,...,23,63,64,74,1,5,57,7,11,60
norm_name,nthroot00data,norm51csv,norm31datacsv,norm41datacsv,norm41csv,nthroot0931data,nthroot08621,nthroot0931,norm31csv,nthroot06207,...,nthroot04138data,norm21datacsv,norm21csv,norm5csv,nthroot003448data,nthroot01034data,norm1datacsv,nthroot01379data,nthroot02069data,norm1csv
accuracy,55.715,68.043,68.85,68.96,68.997,69.382,69.584,69.602,69.712,70.006,...,80.407,80.426,80.444,80.499,80.517,80.646,80.646,80.646,80.664,80.866


In [22]:
accura.nlargest(10, 'accuracy')

,norm_name,accuracy
60,norm1csv,80.866
11,nthroot02069data,80.664
5,nthroot01034data,80.646
57,norm1datacsv,80.646
7,nthroot01379data,80.646
1,nthroot003448data,80.517
74,norm5csv,80.499
64,norm21csv,80.444
63,norm21datacsv,80.426
23,nthroot04138data,80.407


<u>***Note:*** Each time you run the classifier will result in different accuracies.</u>

### As you can see, all 5 normalization techniques do much better than the un-normalized data, with some performing better than others.

### Things you can try, to see how they affect the classifier accuracy:
- Play around with different values of `test_size`. What does testing on more or less data do?
- Play around with different parameters that are passed to `do_classify`. Start with `nn_count` and `embed_dim`(For what those arguments are, and a full list of all of the arguments you can pass to do_classify, look at the function `do_classify` in `/MuyGPyS/examples/classify.py`).
- Try generating more cutouts using `generating_ZTF_cutouts_from_ra_dec.ipynb`. How does having more testing and training data affects the classifier?
- Play around with the parameters used to make the cutouts. What happens if you remove blend cuts? Can the classifier classify blends? What is you increase the seeing limit? Can the classifier classify images with bad atmoshperic quality?

<hr style="border:2px solid gray"> </hr>

## <u>**Optional Step:**</u>
### Running each dataset through the classifier multiple times, testing and training on varying amounts of data, different random states, and plotting the accuracy outcomes

- Each time you run the following steps, you change:
    - `test_size`: This is used in `train_test_split`, and changes the size of the testing and training datasets, which effects the accuracy of the classifier.
    - `random_state`: This is used in `train_test_split`, and changes the ratio of how many stars-to-galaxies get tested on.
- You can set how many times to run the classifier with varying test sizes and random states by setting `num_runs`, and you can manually change the test_size values by editing `test_size_values`.

In [23]:
test_size_values = [.2, .25, .33, .4, .5, .75]
num_runs = 3

In [24]:
# def run_classifier(image_data, truth_labels, test_size, state):
#     X_train, X_test, y_train, y_test = train_test_split(image_data, truth_labels, test_size=test_size, random_state=state)
#     onehot_train, onehot_test = generate_onehot_value(y_train), generate_onehot_value(y_test)
#     train = {'input': X_train, 'output': onehot_train, 'lookup': y_train}
#     test = {'input': X_test, 'output': onehot_test, 'lookup': y_test}
#     #Switch verbose to True for more output
#     muygps, nbrs_lookup, surrogate_predictions= do_classify(
#                         test_features=np.array(test['input']),
#                         train_features=np.array(train['input']), 
#                         train_labels=np.array(train['output']), 
#                         nn_count=20,
#                         batch_count=200,
#                         loss_fn=cross_entropy_fn,
#                         opt_fn=Bayes_optimize,
#                         k_kwargs=k_kwargs_mattern,
#                         nn_kwargs=nn_kwargs_hnsw, 
#                         verbose=False) 
#     predicted_labels = np.argmax(surrogate_predictions, axis=1)
#     accuracy = (np.sum(predicted_labels == np.argmax(test["output"], axis=1))/len(predicted_labels))*100
#     return accuracy

In [25]:
# from time import perf_counter
# start = perf_counter()

# accuracies = pd.DataFrame({'test_size': test_size_values})

# # Setting progress bar for each time the classifier will be run during this step
# pbar = tqdm(total=len(norm_data_names)*num_runs*len(test_size_values), desc='Running classifier', leave=True)

# for path in norm_data_names:
#     path1 = '../data/data-norm/max-pixel-all/' + path
#     data = pd.read_csv(path1,na_values='-')
#     data.fillna(0,inplace=True)
#     data_label = ''.join(path.split('.')[:2])
#     truth_labels = data.iloc[:, 0].values
#     image_data = data.iloc[:, 1:].values
#     all_acc_dataset = []
#     for test_size in test_size_values:
#         acc = []
#         idx = 1
#         while idx <= num_runs:
#             accuracy = run_classifier(image_data, truth_labels, test_size, state=random.randint(0, 10000))
#             acc.append(accuracy)
#             pbar.update(1)
#             idx += 1
#         avg_acc = np.average(acc)
#         all_acc_dataset.append(avg_acc)
#     temp_df = pd.DataFrame({str(data_label): all_acc_dataset})
#     accuracies = pd.concat([accuracies, temp_df], axis=1)
# end = perf_counter()
# print(f"Time taken to run the classifier on all datasets: {(end-start)/60} minutes")
# accuracies.to_csv('max-all-accuracies.csv', index=False)
# display(accuracies)

In [26]:
# plt.figure(figsize=(12,4))

# for path in norm_data_names:
#     path1 = '../data/data-norm/max-pixel-all/' + path
#     data = pd.read_csv(path1,na_values='-')
#     data.fillna(0,inplace=True)
#     data_label = ''.join(path.split('.')[:2])
#     # data_label = 'Normalized {} {}'.format(*path.split('_')[:2])
#     plt.plot(accuracies['test_size'].values, accuracies[data_label].values, label=data_label)

# plt.title("MuyGPs Stellar Blending 2-class")    
# plt.legend(fontsize=10)   
# plt.tick_params(labelsize=10)
# plt.xlabel("Test size (as a ratio to full data size)", fontsize=10)
# plt.ylabel("Accuracy [%]", fontsize=10)
# plt.savefig("muygps_max_all_abs.png")
# plt.show()

In [27]:
# accuracies = pd.read_csv('max-all-accuracies.csv')
# np.max(accuracies.values, axis=1)

In [28]:
# idcs = np.argmax(accuracies.values, axis=1)
# accuracies.iloc[:, idcs]

There is no benefit to normalization images with a division by maximum over the entire data. All the max accuracies above occur where the max-all normalization is not applied. However, it seems neural net model is in favor of max-all normalization instead of image by image max normalization.